In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
%matplotlib inline

## Analysing data

In [2]:
data, labels = [], []

label_map = {
    "Iris-setosa": 0, 
    "Iris-versicolor": 1, 
    "Iris-virginica": 2
}

with open(os.path.join('iris','iris.data'),'r') as f:
    for row in f:
        if len(row.strip())>0:
            tokens = row.split(',')
            data.append(tokens[0:4])
            labels.append(label_map[tokens[4].strip()])
        
print("Sample data")
print(data[:5])
print(labels[:5])


# Convert to numpy array
data = np.array(data)
labels = np.array(labels)

onehot_labels = np.zeros(shape=(data.shape[0],3))
onehot_labels[np.arange(0,data.shape[0]),labels]=1.0

print("Shapes of datasets")
print(data.shape)
print(labels.shape)
print(onehot_labels.shape)

Sample data
[['5.1', '3.5', '1.4', '0.2'], ['4.9', '3.0', '1.4', '0.2'], ['4.7', '3.2', '1.3', '0.2'], ['4.6', '3.1', '1.5', '0.2'], ['5.0', '3.6', '1.4', '0.2']]
[0, 0, 0, 0, 0]
Shapes of datasets
(150, 4)
(150,)
(150, 3)


## Define the graph of a fully-connected network

In [3]:
tf.reset_default_graph()

# Input
tf_in = tf.placeholder(name='in', shape=[25,4], dtype=tf.float32)

# First layer
fulcon1 = tf.get_variable('fulcon1', shape=[4, 100], initializer=tf.glorot_uniform_initializer)

# Output and non-linearity
tf_out1 = tf.matmul(tf_in, fulcon1)
tf_out1 = tf.nn.leaky_relu(tf_out1)
print(tf_out1.shape)
# Second layer
fulcon2 = tf.get_variable('fulcon2', shape=[100,3], initializer=tf.glorot_uniform_initializer)

# Second layer output
tf_out2 = tf.matmul(tf_out1, fulcon2)
print(tf_out2.shape)
# Final prediction
tf_pred = tf.nn.softmax(tf_out2)

(25, 100)
(25, 3)


## Defining loss and optimizer

In [4]:
tf_label =  tf.placeholder(name='out', shape=[25, 3], dtype=tf.float32)

# Loss
tf_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_label, logits=tf_out2)

# Optimizer
tf_opt = tf.train.MomentumOptimizer(learning_rate=1e-5, momentum=0.9).minimize(tf_loss)

## Splitting the dataset to train and test

In [8]:
np.random.seed(100)

data_mask = np.random.choice([True,False], size=150, p=[0.8, 0.2])

train_data = data[data_mask,:]
train_labels = labels[data_mask]
train_onehot_labels = onehot_labels[data_mask, :]

test_data = data[~data_mask, :]
test_labels = labels[~data_mask]
test_onehot_labels = onehot_labels[~data_mask, :]

print('Dataset sizes')
print(train_data.shape)
print(test_data.shape)

#print(train_data)
print(train_data[:5,:])

Dataset sizes
(119, 4)
(31, 4)
[['5.1' '3.5' '1.4' '0.2']
 ['4.9' '3.0' '1.4' '0.2']
 ['4.7' '3.2' '1.3' '0.2']
 ['5.0' '3.6' '1.4' '0.2']
 ['5.4' '3.9' '1.7' '0.4']]


## Running the neural network

In [11]:
def compute_accuracy(labels, predictions):
    
    true_classes = np.argmax(labels, axis=1).ravel()
    pred_classes = np.argmax(predictions, axis=1).ravel()
    
    accuracy = np.sum((true_classes==pred_classes))/true_classes.shape
    return accuracy

In [12]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    
    # Initialize global variables
    tf.global_variables_initializer().run()
    
    # Running number of epochs
    for epoch in range(100):
        losses = []
        for bi in range(0, train_data.shape[0]-25, 25):
                         
            batch_data = train_data[bi:bi+25, :]
            batch_labels = train_onehot_labels[bi:bi+25, :]

            if batch_data.shape[0] != 25:
                continue
                
            _, l = sess.run([tf_opt, tf_loss], feed_dict={tf_in: batch_data, tf_label: batch_labels})

            losses.append(l)
        
        """ Testing phase """
        accuracy = []
        for bi in range(0, test_data.shape[0]-25, 25):
            
            batch_data = test_data[bi:bi+25,:]
            batch_labels = test_onehot_labels[bi:bi+25,:]
            
            test_pred = sess.run(tf_pred, feed_dict={tf_in:batch_data, tf_label: batch_labels})
            b_acc = compute_accuracy(labels=batch_labels, predictions=test_pred)
            accuracy.append(b_acc)
        print('Loss for epoch {} is: {}'.format(epoch, np.mean(losses)))
        print('Test accuracy: {}'.format(np.mean(accuracy)))

Loss for epoch 0 is: 2.4657678604125977
Test accuracy: 0.48
Loss for epoch 1 is: 2.3096506595611572
Test accuracy: 0.48
Loss for epoch 2 is: 2.0721378326416016
Test accuracy: 0.48
Loss for epoch 3 is: 1.8117997646331787
Test accuracy: 0.44
Loss for epoch 4 is: 1.5671981573104858
Test accuracy: 0.0
Loss for epoch 5 is: 1.36037278175354
Test accuracy: 0.0
Loss for epoch 6 is: 1.2058345079421997
Test accuracy: 0.16
Loss for epoch 7 is: 1.1043022871017456
Test accuracy: 0.12
Loss for epoch 8 is: 1.0473767518997192
Test accuracy: 0.32
Loss for epoch 9 is: 1.020133376121521
Test accuracy: 0.32
Loss for epoch 10 is: 1.0072144269943237
Test accuracy: 0.64
Loss for epoch 11 is: 0.9978235363960266
Test accuracy: 0.8
Loss for epoch 12 is: 0.9867751598358154
Test accuracy: 0.8
Loss for epoch 13 is: 0.9731189012527466
Test accuracy: 0.8
Loss for epoch 14 is: 0.9579187035560608
Test accuracy: 0.8
Loss for epoch 15 is: 0.9426800608634949
Test accuracy: 0.8
Loss for epoch 16 is: 0.9285731315612793
Tes